# Housing Prices


This is my second attempt at Housing Prices. I got pretty much lost in the first one but I learnt quite a few things. Even though I had poor results, it was good for skillup.


In [1]:
#Basic libraries first
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
#loadint data
data=pd.read_csv("../Housing Prices/train.csv")
testdata=pd.read_csv("../Housing Prices/test.csv")
data.shape

(1460, 81)

In [ ]:
# test.head()

In [ ]:
# Fill nas for plot- just the train data. not part of final solution
#lets fill nans
# data.isna().sum()
cont_features=[]
cat_features=[]
data.drop('Id',axis=1,inplace=True)
data.GarageType.fillna('None')
for col in data.columns:
    if data[col].dtype=='int64' or data[col].dtype=='float64':
        cont_features.append(col)
        data[col].fillna(data[col].median(),inplace=True)
    else:
        cat_features.append(col)
        data[col].fillna(data[col].value_counts().idxmax(),inplace=True)

In [ ]:
#plots I used extensively for feature engineering, feature selection
# a = sns.FacetGrid(data=data,col=cont_features)
for col in cont_features:
    fig,ax=plt.subplots(figsize=(5,5))
    sns.regplot(x=col,y='SalePrice', data=data)

In [ ]:
#plots I used extensively for feature engineering, feature selection
for col in cat_features:
#     fig,ax=plt.subplots(figsize=(5,5))
    sns.catplot(x=col,y='SalePrice', data=data,kind="box")

In [ ]:
#Map I used extensively for feature engineering, feature selection
fig, ax=plt.subplots(figsize=(40,40))
sns.heatmap(data.corr(),annot=True)

In [3]:
#I got these using visual analysis of data
cont_features2=['SalePrice','LotFrontage','MasVnrArea', 'TotalBsmtSF', '1stFlrSF','2ndFlrSF', 'GrLivArea'] 
cat_features2= ['YearBuilt','YearRemodAdd','BsmtFullBath', 'BsmtHalfBath','FullBath','HalfBath','OverallQual','TotRmsAbvGrd', 'Fireplaces','MSZoning','Street','LotShape','LotConfig','Condition1',
        'BldgType','Neighborhood','HouseStyle',
        'BsmtFinType1','Electrical','FireplaceQu','GarageType','SaleType','SaleCondition']

In [4]:
def skewdetfix(feat):
    sk=feat.skew()
    if sk>1 or sk<1:
#         fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)
#         feat.fillna(0)
#         sns.distplot(feat, ax=ax1)
        feat=np.log1p(feat)
#         feat.fillna(0)
#         sns.distplot(feat,ax=ax2,color="k")
    return feat

In [5]:
from numpy import percentile
# print(data.GrLivArea.shape)


def outlierdetectfix(feat):
#     print(feat)
    q25,q75=percentile(feat,25),percentile(feat,75)
    iqr=(q75-q25)
    iqr = q75 - q25
#     type(feat)
    lower = q25 - (iqr*1.5)
    upper = q75 + (iqr*1.5)
#     print(upper)
    feat.loc[feat>upper] = upper
    return feat


# data.GrLivArea=oulierdetectfix(data.GrLivArea)

In [6]:
for col in cont_features2:
    data[col]=skewdetfix(data[col])
    data[col]=outlierdetectfix(data[col])

/home/charlie/environments/PFDS/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/charlie/environments/PFDS/lib/python3.6/site-packages/numpy/lib/function_base.py:4291: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


In [7]:
# testdata['SalePrice']=1
y=data[['SalePrice']].copy()
data.drop('SalePrice',axis=1,inplace=True)
data2=pd.concat([data,testdata],axis=0)
data2.shape

(2919, 80)

In [8]:
#fill nas
data2.GarageType.fillna('None')
for col in data2.columns:
    if data2[col].dtype=='int64' or data2[col].dtype=='float64':
#         cont_features.append(col)
        data2[col].fillna(data2[col].median(),inplace=True)
    else:
#         cat_features.append(col)
        data2[col].fillna(data2[col].value_counts().idxmax(),inplace=True)

In [9]:
#drop extra cols
for col in data2.columns:
#     print (col)
    if col not in cont_features2 and col not in cat_features2:
        data2.drop(col, axis=1, inplace=True)
        

In [10]:
#feature engineering
data2['Make']=data2[['YearBuilt','YearRemodAdd']].max(axis=1)
data2.loc[ data2['Make'] <= 1950, 'Make'] = 1950
data2.loc[(data2['Make'] > 1950) & (data2['Make']<= 1960), 'Make'] = 1960
data2.loc[(data2['Make'] > 1960) & (data2['Make']<= 1970), 'Make'] = 1970
data2.loc[(data2['Make'] > 1970) & (data2['Make']<= 1980), 'Make'] = 1980
data2.loc[(data2['Make'] > 1980) & (data2['Make']<= 1990), 'Make'] = 1990
data2.loc[(data2['Make'] > 1990) & (data2['Make']<= 2000), 'Make'] = 2000
data2.loc[(data2['Make'] > 2000) & (data2['Make']<= 2010), 'Make'] = 2010
data2.loc[ data2['Make'] > 2010, 'Make'] = 6



In [11]:
data2.drop(['YearBuilt','YearRemodAdd'],axis=1,inplace=True)

In [12]:
data2['TotalFSF']=data2['1stFlrSF']+data2['2ndFlrSF']
data2.drop(['1stFlrSF','2ndFlrSF'],axis=1,inplace=True)

In [13]:
data2['TotalBath']=data2['BsmtFullBath']+data2['BsmtHalfBath']+data2['FullBath']+data2['HalfBath']
data2.drop(['BsmtFullBath', 'BsmtHalfBath','FullBath','HalfBath'],axis=1,inplace=True)


I used plots to get an idea of the relationships and the variables that seem necessary. I fixed the outliers in training data before anything else so that I can combine data and then do missing values, feature engineering and encoding on all the data together. Since the data isnt neccesarily ordinal, I will use onehot encode and then do PCA of the whole set together. That way I can try out different models without worrying about cat features. I will do a correlation check before to check if I am taking the right features. I might do another corr after encoding. lets see how this goes.

In [14]:
data2.LotShape.replace(['IR2','IR3'], 'other', inplace=True)
data2.Condition1.replace(['Feedr','Artery','RRAn','PosN','RRAe','PosA','RRNn','RRNe'],'other',inplace=True)

data2.Electrical.replace(['FuseA','FuseF','FuseP'],'other',inplace=True)

data2.GarageType.replace(['BuiltIn','Basment','CarPort','2Types'],'other',inplace=True)
data2.SaleType.replace(['COD','ConLD','ConLw','ConLi','CWD','Oth','Con'],'other',inplace=True)

data2.SaleCondition.replace(['Family','Alloca','AdjLand'],'other',inplace=True)

In [15]:
#encoding and scaling

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

In [16]:
data2.drop('Neighborhood',axis=1,inplace=True)

In [17]:

cont_features3=['LotFrontage', 'MasVnrArea', 'TotalBsmtSF', 'TotalFSF', 'GrLivArea'] 
cat_features3=[ 'Make','TotalBath','OverallQual','TotRmsAbvGrd', 'Fireplaces','MSZoning','Street','LotShape','LotConfig','Condition1',
        'BldgType','HouseStyle',
        'BsmtFinType1','Electrical','FireplaceQu','GarageType','SaleType','SaleCondition']

In [18]:
def encoder(var):
    return var.astype("category")
    

In [19]:
for col in cat_features3:
    data2[col]=encoder(data2[col])

In [20]:
dummies=pd.get_dummies(data2[cat_features3])
# dummies

In [21]:
# data2.drop(cat_features3,axis=1,inplace=True)
data2=pd.concat([data2,dummies],axis=1)

In [22]:
data2.drop(cat_features3,axis=1,inplace=True)
# data2

In [23]:
train=data2.iloc[:1460,:]
test=data2.iloc[1460 : ,:]



In [24]:
# Instantiate encoder/scaler
scaler = MinMaxScaler([0,1])
scaler.fit(train[cont_features3]) 
train[cont_features3]=scaler.transform(train[cont_features3]) 
test[cont_features3]=scaler.transform(test[cont_features3]) 

/home/charlie/environments/PFDS/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/charlie/environments/PFDS/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/charlie/environments/PFDS/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [27]:
# import numpy as np
from sklearn.decomposition import PCA
# X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
pca = PCA(n_components=60)
train2=pca.fit_transform(train)


var=pca.explained_variance_ratio_
var1=np.cumsum(np.round(var,decimals=4)*100)

print(var1)  


[11.93 19.34 25.79 30.71 34.97 38.66 41.91 45.03 48.06 50.97 53.75 56.38
 58.95 61.28 63.46 65.49 67.41 69.15 70.84 72.35 73.83 75.21 76.58 77.86
 79.07 80.26 81.38 82.47 83.51 84.48 85.43 86.34 87.2  88.04 88.85 89.6
 90.34 91.04 91.72 92.36 92.94 93.48 94.   94.47 94.92 95.34 95.75 96.15
 96.53 96.87 97.17 97.45 97.7  97.94 98.14 98.33 98.5  98.66 98.81 98.96]


In [31]:
pca = PCA(n_components=50)
train_PCA=pca.fit(train)
train_PCA=pca.transform(train)
test_PCA=pca.transform(test)


In [46]:
y_train=y.iloc[:1460,:]
y_test=data2.iloc[1460 : ,:]

In [47]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_trn,y_tst=train_test_split(train_PCA,y_train,test_size=0.33,random_state=45)

In [51]:
x_train

array([[ 0.71230103,  0.91124651,  0.61100757, ...,  0.04207268,
         0.09609671,  0.12164085],
       [ 0.21947424, -0.24042419, -0.75601532, ...,  0.29149439,
         0.15922704, -0.15188291],
       [-0.80775444, -0.76163501,  0.21082279, ...,  0.03601982,
         0.05428591, -0.03309107],
       ...,
       [ 0.01385213,  0.35790724,  0.77899443, ..., -0.0650179 ,
        -0.12424245, -0.01888905],
       [ 1.57625333,  0.8296774 , -0.81030444, ..., -0.33174517,
        -0.09866577, -0.0391767 ],
       [-0.20478006,  0.40306654,  1.12122459, ...,  0.28574781,
        -0.24367592, -0.5074157 ]])

In [37]:
#randomforest test
from sklearn.ensemble import RandomForestClassifier

# rf2.score(x_test,y_test)

In [38]:
rf2=RandomForestClassifier(n_estimators= 3000,min_samples_split= 5,min_samples_leaf= 2)
# rf2.fit(x_train,y_train)

In [48]:
rf2.fit(x_train,y_trn)

/home/charlie/environments/PFDS/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


ValueError: Unknown label type: 'continuous'

In [ ]:
# cat_columns_idx = [data2.columns.get_loc(col) 
#                    for col in cat_features3]

In [ ]:
# ohe    = OneHotEncoder(categorical_features=cat_columns_idx,sparse=False,handle_unknown='ignore',dtype='int')
# # #  for col in cat_features3:
# dummies=    ohe.fit_transform(data2)

In [ ]:
# from sklearn.preprocessing import LabelBinarizer

# lb = LabelBinarizer()
# for col in cat_features3:
#     feature=lb.fit_transform(data2[col])
#     n_feature=pd.concat([n_feature,pd.DataFrame(feature)
# #     data3=pd.concat([data2,n_feature],axis=1)
#     print(n_feature.head(10))

In [ ]:
data2[cat_features3]

In [ ]:
dumm=pd.DataFrame(dummies)

dumm.head().T

In [ ]:
# def skewdetfix(feat):
#     sk=feat.skew()
#     if sk>1 or sk<1:
#         feat=np.log(feat)
#     return feat
# # data.SalePrice.skew()

In [ ]:
# from scipy.stats import skew
# skew(data.SalePrice)
# data.SalePrice=np.log(data.SalePrice)
# data.SalePrice.skew()

In [ ]:
#just checking relation between garrage building and others 
# sns.relplot(x="GarageYrBlt",y="YearRemodAdd",data=data)

In [ ]:
# #function for outliers on train data
# #  from scipy.stats import iqr
# # def outlierremove(ser)
# from numpy import percentile
# # print(data.GrLivArea.shape)


# def oulierdetectfix(feat):
# #     print(feat)
#     q25,q75=percentile(feat,25),percentile(feat,75)
#     iqr=(q75-q25)
#     iqr = q75 - q25
# #     type(feat)
#     lower = q25 - (iqr*3)
#     upper = q75 + (iqr*3)
# #     print(upper)
#     feat.loc[feat>upper] = upper
# #     return feata
# data.GrLivArea=oulierdetectfix(data.GrLivArea)
# # print(data.GrLivArea.shape)
# # data.GrLivArea=oulierdetectfix(data.GrLivArea)
# # data.TotalBsmtSF[data.TotalBsmtSF<upper]

In [ ]:
#function for outliers on train data
#  from scipy.stats import iqr
# def outlierremove(ser)

# print(data.GrLivArea.shape)
# data.TotalBsmtSF[data.TotalBsmtSF<upper]

In [ ]:
#apply outlier


In [ ]:
#logical fix missing data

In [ ]:
#function for missing all data

In [ ]:
#combine the regfeatures 

In [ ]:
#drop unused reg

In [ ]:
#function for skew detect and fix all data

In [ ]:
#drop unused cat data

In [ ]:
#reduce the items in data logically

In [ ]:
#hot encode

In [ ]:
#Scale

In [ ]:
#PCA

In [ ]:
#model XGboost

In [ ]:
#model lasso

In [ ]:
#ensemble